<a href="https://colab.research.google.com/github/dylansy/ISCS_30.65/blob/main/%5BSubmission_4%5D_Autoencoders_with_Inception_Module_and_Residual_Skip_Connections.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## CNN Based Autoencoder

Source: https://www.geeksforgeeks.org/implement-convolutional-autoencoder-in-pytorch-with-cuda/



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import numpy as np
from PIL import Image

In [ ]:

import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
import numpy as np
from PIL import Image
import os

class ResidualBlock(nn.Module):
    def __init__(self, channels):
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Conv2d(channels, channels, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(channels, channels, kernel_size=3, stride=1, padding=1)
        self.relu = nn.ReLU()

    def forward(self, x):
        residual = x
        out = self.relu(self.conv1(x))
        out = self.conv2(out)
        out += residual
        out = self.relu(out)
        return out

class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            ResidualBlock(16),
            nn.MaxPool2d(kernel_size=2, stride=2),
            ResidualBlock(16)
        )
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(16, 16, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.ReLU(),
            ResidualBlock(16),
            nn.ConvTranspose2d(16, 3, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

# Define PSNR calculation function
def calculate_psnr(original, denoised):
    mse = np.mean((original - denoised) ** 2)
    max_pixel_value = 1.0  # Assuming images are normalized between 0 and 1

    psnr = 10 * np.log10((max_pixel_value ** 2) / mse)
    return psnr

# Define transform
transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
])

# Move the model to GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Train an autoencoder for each folder
for folder_name in ['25', '50', '75']:
    # Load dataset from 'noisy_train' and 'noisy_test' folders
    train_dataset = datasets.ImageFolder(root=f'/content/drive/MyDrive/Colab Notebooks/pneumonia/noisy_train/{folder_name}', transform=transform)
    test_dataset = datasets.ImageFolder(root=f'/content/drive/MyDrive/Colab Notebooks/pneumonia/noisy_test/{folder_name}', transform=transform)

    # Define the dataloader
    train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=128, shuffle=True)
    test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=128)

    # Initialize the autoencoder
    model = Autoencoder().to(device)

    # Define the loss function and optimizer
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    # Train the autoencoder
    num_epochs = 50
    for epoch in range(num_epochs):
        for data in train_loader:
            img, _ = data
            img = img.to(device)
            optimizer.zero_grad()
            output = model(img)
            loss = criterion(output, img)
            loss.backward()
            optimizer.step()
        if epoch % 5 == 0:
            print(f'Folder {folder_name}, Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

    # Evaluate PSNR on the test dataset
    psnr_total = 0.0
    num_samples = 0

    for i, (img, paths) in enumerate(test_loader):
        img = img.to(device)
        output = model(img)

        # Iterate over batch images
        for i in range(img.size(0)):
            index = num_samples + i
            if index < len(test_dataset.samples):  # Check if the index is within the range
                original_img_path = os.path.join('/content/drive/MyDrive/Colab Notebooks/pneumonia/test', os.path.basename(test_dataset.samples[index][0]))
                original_img = np.array(Image.open(original_img_path).convert('RGB')) / 255.0  # Normalize to [0, 1]
                original_img_tensor = torch.tensor(original_img).permute(2, 0, 1).unsqueeze(0).float().to(device)

                # Resize the output to match the original image size
                resized_output = torch.nn.functional.interpolate(output[i].unsqueeze(0), size=original_img_tensor.shape[-2:], mode='bilinear', align_corners=False)

            # Calculate PSNR
            psnr = calculate_psnr(original_img_tensor.cpu().numpy(), resized_output.detach().cpu().numpy())
            psnr_total += psnr
            num_samples += 1

    average_psnr = psnr_total / num_samples
    print('Average PSNR on test dataset: {:.4f} dB'.format(average_psnr))

    # Save the model
    torch.save(model.state_dict(), f'conv_autoencoder_{folder_name}.pth')

Folder 25, Epoch [1/50], Loss: 0.0730
Folder 25, Epoch [6/50], Loss: 0.0551
Folder 25, Epoch [11/50], Loss: 0.0486
Folder 25, Epoch [16/50], Loss: 0.0358
Folder 25, Epoch [21/50], Loss: 0.0305
Folder 25, Epoch [26/50], Loss: 0.0256
Folder 25, Epoch [31/50], Loss: 0.0186
Folder 25, Epoch [36/50], Loss: 0.0217
Folder 25, Epoch [41/50], Loss: 0.0185
Folder 25, Epoch [46/50], Loss: 0.0157
Average PSNR on test dataset: 12.8993 dB
Folder 50, Epoch [1/50], Loss: 0.0971
Folder 50, Epoch [6/50], Loss: 0.0758
Folder 50, Epoch [11/50], Loss: 0.0595
Folder 50, Epoch [16/50], Loss: 0.0398
Folder 50, Epoch [21/50], Loss: 0.0246
Folder 50, Epoch [26/50], Loss: 0.0195
Folder 50, Epoch [31/50], Loss: 0.0184
Folder 50, Epoch [36/50], Loss: 0.0136
Folder 50, Epoch [41/50], Loss: 0.0138
Folder 50, Epoch [46/50], Loss: 0.0124
Average PSNR on test dataset: 11.5892 dB
Folder 75, Epoch [1/50], Loss: 0.1275
Folder 75, Epoch [6/50], Loss: 0.0751
Folder 75, Epoch [11/50], Loss: 0.0431
Folder 75, Epoch [16/50], L

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
import numpy as np
from PIL import Image
import os

class InceptionModule(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(InceptionModule, self).__init__()
        self.branch1x1 = nn.Conv2d(in_channels, out_channels, kernel_size=1)

        self.branch3x3_1 = nn.Conv2d(in_channels, out_channels, kernel_size=1)
        self.branch3x3_2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1)

        self.branch5x5_1 = nn.Conv2d(in_channels, out_channels, kernel_size=1)
        self.branch5x5_2 = nn.Conv2d(out_channels, out_channels, kernel_size=5, padding=2)

        self.branch_pool = nn.Conv2d(in_channels, out_channels, kernel_size=1)

    def forward(self, x):
        branch1x1 = self.branch1x1(x)

        branch3x3 = self.branch3x3_1(x)
        branch3x3 = self.branch3x3_2(branch3x3)

        branch5x5 = self.branch5x5_1(x)
        branch5x5 = self.branch5x5_2(branch5x5)

        branch_pool = nn.functional.avg_pool2d(x, kernel_size=3, stride=1, padding=1)
        branch_pool = self.branch_pool(branch_pool)

        outputs = [branch1x1, branch3x3, branch5x5, branch_pool]
        return torch.cat(outputs, 1)

class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            InceptionModule(3, 16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            InceptionModule(64, 8),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(32, 8, kernel_size=3, stride=2, padding=1, output_padding=1),  # Modify this line
            nn.ReLU(),
            nn.ConvTranspose2d(8, 16, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.ReLU(),
            nn.Conv2d(16, 3, kernel_size=3, stride=1, padding=1),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

# Define PSNR calculation function
def calculate_psnr(original, denoised):
    mse = np.mean((original - denoised) ** 2)
    max_pixel_value = 1.0  # Assuming images are normalized between 0 and 1

    psnr = 10 * np.log10((max_pixel_value ** 2) / mse)
    return psnr

# Define transform
transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
])

# Move the model to GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Train an autoencoder for each folder
for folder_name in ['25', '50', '75']:
    # Load dataset from 'noisy_train' and 'noisy_test' folders
    train_dataset = datasets.ImageFolder(root=f'/content/drive/MyDrive/Colab Notebooks/pneumonia/noisy_train/{folder_name}', transform=transform)
    test_dataset = datasets.ImageFolder(root=f'/content/drive/MyDrive/Colab Notebooks/pneumonia/noisy_test/{folder_name}', transform=transform)

    # Define the dataloader
    train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=128, shuffle=True)
    test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=128)

    # Initialize the autoencoder
    model = Autoencoder().to(device)

    # Define the loss function and optimizer
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    # Train the autoencoder
    num_epochs = 50
    for epoch in range(num_epochs):
        for data in train_loader:
            img, _ = data
            img = img.to(device)
            optimizer.zero_grad()
            output = model(img)
            loss = criterion(output, img)
            loss.backward()
            optimizer.step()
        if epoch % 5 == 0:
            print(f'Folder {folder_name}, Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

    # Evaluate PSNR on the test dataset
    psnr_total = 0.0
    num_samples = 0

    for i, (img, paths) in enumerate(test_loader):
        img = img.to(device)
        output = model(img)

        # Iterate over batch images
        for i in range(img.size(0)):
            index = num_samples + i
            if index < len(test_dataset.samples):  # Check if the index is within the range
                original_img_path = os.path.join('/content/drive/MyDrive/Colab Notebooks/pneumonia/test', os.path.basename(test_dataset.samples[index][0]))
                original_img = np.array(Image.open(original_img_path).convert('RGB')) / 255.0  # Normalize to [0, 1]
                original_img_tensor = torch.tensor(original_img).permute(2, 0, 1).unsqueeze(0).float().to(device)

                # Resize the output to match the original image size
                resized_output = torch.nn.functional.interpolate(output[i].unsqueeze(0), size=original_img_tensor.shape[-2:], mode='bilinear', align_corners=False)

            # Calculate PSNR
            psnr = calculate_psnr(original_img_tensor.cpu().numpy(), resized_output.detach().cpu().numpy())
            psnr_total += psnr
            num_samples += 1

    average_psnr = psnr_total / num_samples
    print('Average PSNR on test dataset: {:.4f} dB'.format(average_psnr))

    # Save the model
    torch.save(model.state_dict(), f'conv_autoencoder_{folder_name}.pth')

Folder 25, Epoch [1/50], Loss: 0.0800
Folder 25, Epoch [6/50], Loss: 0.0798
Folder 25, Epoch [11/50], Loss: 0.0669
Folder 25, Epoch [16/50], Loss: 0.0530
Folder 25, Epoch [21/50], Loss: 0.0409
Folder 25, Epoch [26/50], Loss: 0.0276
Folder 25, Epoch [31/50], Loss: 0.0284
Folder 25, Epoch [36/50], Loss: 0.0278
Folder 25, Epoch [41/50], Loss: 0.0180
Folder 25, Epoch [46/50], Loss: 0.0235
Average PSNR on test dataset: 12.6581 dB
Folder 50, Epoch [1/50], Loss: 0.0914
Folder 50, Epoch [6/50], Loss: 0.0827
Folder 50, Epoch [11/50], Loss: 0.0524
Folder 50, Epoch [16/50], Loss: 0.0398
Folder 50, Epoch [21/50], Loss: 0.0236
Folder 50, Epoch [26/50], Loss: 0.0151
Folder 50, Epoch [31/50], Loss: 0.0156
Folder 50, Epoch [36/50], Loss: 0.0185
Folder 50, Epoch [41/50], Loss: 0.0118
Folder 50, Epoch [46/50], Loss: 0.0123
Average PSNR on test dataset: 11.2026 dB
Folder 75, Epoch [1/50], Loss: 0.1060
Folder 75, Epoch [6/50], Loss: 0.1053
Folder 75, Epoch [11/50], Loss: 0.0430
Folder 75, Epoch [16/50], L